In [1]:
!pip install seaborn


In [2]:

%pip install matplotlib
%pip install seaborn
%pip install pandas numpy scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
def balance_check(data, target):
    try:
        df = pd.read_csv(data)
    except:
        print("Error Reading cannot be found")
        return

    #now check value counts
    ones_percentage = df[target].value_counts(normalize=True).loc[1] * 100
    print(f"Percentage of 1s for : {ones_percentage:.2f}%")

    return



In [4]:
balance_check("data/train.csv/train.csv","toxic")
balance_check("data/train.csv/train.csv","severe_toxic")
balance_check("data/train.csv/train.csv","obscene")
balance_check("data/train.csv/train.csv","threat")
balance_check("data/train.csv/train.csv","insult")
balance_check("data/train.csv/train.csv","identity_hate")

Percentage of 1s for : 9.58%
Percentage of 1s for : 1.00%
Percentage of 1s for : 5.29%
Percentage of 1s for : 0.30%
Percentage of 1s for : 4.94%
Percentage of 1s for : 0.88%


In [5]:
df1 = pd.read_csv("data/train.csv/train.csv")
df1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:

%pip install nltk

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

%pip install transformers


Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

# ✅ NLTK Fix for LookupError
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Dynamically resolve the user's nltk_data path
nltk_path = os.path.join(os.path.expanduser('~'), 'AppData', 'Roaming', 'nltk_data')
if nltk_path not in nltk.data.path:
    nltk.data.path.append(nltk_path)

# Trigger a preload of stopwords to catch missing resource issues early
try:
    _ = stopwords.words('english')
except LookupError as e:
    print("NLTK stopwords resource not found, attempting to download again...")
    nltk.download('stopwords')
    _ = stopwords.words('english')

def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path)

        print("Dataset loaded successfully!")
        print(f"Shape: {df.shape}")

        # check for null values
        print("\nNull values per column:")
        print(df.isnull().sum())

        return df

    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # remove artifacts and wikimarkups
    text = re.sub(r'==.*?==+', '', text)
    text = re.sub(r'\[\[.*?\]\]', '', text)
    text = re.sub(r'\{\{.*?\}\}', '', text)
    text = re.sub(r'__[A-Z]+__', '', text)
    text = re.sub(r'&[a-z]+;', '', text)
    text = re.sub(r'<.*?>', '', text)

    # URLs, IPs, timestamps, extra spaces, and lowercasing
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '', text)
    text = re.sub(r'\d{1,2}:\d{2},?\s*\d{1,2}\s+\w+\s+\d{4}', '', text)
    text = re.sub(r"[^\w\s'!?.]", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()

    return text

def remove_stopwords(text):
    try:
        stop_words = set(stopwords.words('english'))

        custom_stopwords = {
            'wikipedia', 'page', 'article', 'edit', 'user', 'talk',
            'section', 'redirect', 'category', 'file', 'image', 'template'
        }
        stop_words.update(custom_stopwords)

        negation_words = {
            'not', 'no', 'never', 'none', 'nor', 'neither',
            "don't", "isn't", "aren't", "wasn't", "weren't",
            "haven't", "hasn't", "hadn't", "won't", "wouldn't"
        }
        stop_words -= negation_words

        tokens = word_tokenize(text)
        filtered_tokens = [
            word for word in tokens 
            if word not in stop_words and len(word) > 2 and not word.startswith("'")
        ]

        return ' '.join(filtered_tokens)
    except LookupError as e:
        if "stopwords" in str(e) or "punkt" in str(e):
            nltk.download('punkt', quiet=True)
            nltk.download('stopwords', quiet=True)
            try:
                stop_words = set(stopwords.words('english'))
                tokens = word_tokenize(text)
                return ' '.join(tokens)  # fallback: just tokenize
            except Exception:
                return text  # worst case: return raw text
        return text

def initialize_bert():
    print("\nInitializing BERT model...")
    try:
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        print("BERT initialized successfully")
        return tokenizer, model
    except Exception as e:
        print(f"Error initializing BERT: {e}")
        return None, None

if __name__ == "__main__":
    FILE_PATH = "data/train.csv/train.csv"

    print("Starting Toxic Comment Classification Pipeline")
    df = load_dataset(FILE_PATH)

    if df is not None:
        df['cleaned_text'] = df['comment_text'].apply(clean_text)
        df['processed_text'] = df['cleaned_text'].apply(remove_stopwords)

        # show some examples of cleaning
        print("\nText processing examples:")
        for i in range(3):
            print(f"\nOriginal {i}:")
            print(df['comment_text'].iloc[i])
            print("\nProcessed:")
            print(df['processed_text'].iloc[i])
            print("-"*50)

        bert_tokenizer, bert_model = initialize_bert()

        if bert_tokenizer:
            print("\nTokenization example:")
            sample_text = df['processed_text'].iloc[0]
            print(f"\nSample text: {sample_text}")

            tokens = bert_tokenizer.tokenize(sample_text)
            print(f"\nTokens: {tokens}")

            token_ids = bert_tokenizer.encode(sample_text, add_special_tokens=True)
            print(f"\nToken IDs: {token_ids}")

    print("\nPipeline execution complete")


c:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Starting Toxic Comment Classification Pipeline
Dataset loaded successfully!
Shape: (159571, 8)

Null values per column:
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

Text processing examples:

Original 0:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Processed:
explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.
--------------------------------------------------

Original 1:
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk)

c:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\angel\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develop

BERT initialized successfully

Tokenization example:

Sample text: explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.

Tokens: ['explanation', 'why', 'the', 'edit', '##s', 'made', 'under', 'my', 'user', '##name', 'hardcore', 'metallic', '##a', 'fan', 'were', 'reverted', '?', 'they', 'weren', "'", 't', 'van', '##dal', '##isms', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fa', '##c', '.', 'and', 'please', 'don', "'", 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'i', "'", 'm', 'retired', 'now', '.']

Token IDs: [101, 7526, 2339, 1996, 10086, 2015, 2081, 2104, 2026, 5310, 18442, 13076, 12392, 2050, 5470, 2020, 16407, 1029, 2027, 4694, 1005, 1056, 3158, 9305, 22556, 2074, 8503, 2006, 2070, 3806, 2044, 1045, 5444, 20

In [8]:
if df is not None:
    df['cleaned_text'] = df['comment_text'].apply(clean_text)
    df['processed_text'] = df['cleaned_text'].apply(remove_stopwords)


In [9]:
import torch
from tqdm import tqdm
tqdm.pandas()

def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
    return cls_embedding.squeeze().numpy()

df['bert_embedding'] = df['processed_text'].progress_apply(
    lambda x: get_bert_embedding(x, bert_tokenizer, bert_model)
)


  0%|          | 0/159571 [00:00<?, ?it/s]

100%|██████████| 159571/159571 [3:47:46<00:00, 11.68it/s]  
